In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

from sklearn import linear_model, preprocessing
from sklearn.cluster import KMeans

from keras.models import Sequential
from keras.layers import Dense 
# fix random seed for reproducibility
np.random.seed(7)
path = os.getcwd()

data = pd.read_csv(path + '/GEFCOM/mod_data.csv')

Using TensorFlow backend.
/Users/charlesnatoli/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [171]:
def assign_closest_day(data_input, day,neighb = 20) :  
    data_subset = data_input[data_input['totDay'] > day['totDay'] - neighb]
    data_subset = data_subset[data_subset['totDay'] < day['totDay'] + neighb]

    data_subset = data_subset[data_subset['month'] ==  day['month']]
    data_subset = data_subset[data_subset['hour'] ==  day['hour']]

    n_days = data_subset.shape[0]

    dists = np.zeros(n_days)

    d = (day[['w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11']] - 
                data_subset[['w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11']]) 
    dists = np.sum(d ** 2,axis = 1)
    
    match_day = np.argmin(dists)
   
    return(data_subset.loc[match_day]) 
    
    
def find_all_similar_days_test(input_data_zone,test_data) :

    similar_days = []
    input_data_zone.index = range(input_data_zone.shape[0])
    for i in range(test_data.shape[0]) :
        similar_days.append(assign_closest_day(input_data_zone,test_data.iloc[i],100))

        if i % 2500 == 0:
            print(str(datetime.now()),i)

    similar_days_df = pd.DataFrame(similar_days)
    similar_days_df.index = range(similar_days_df.shape[0])
    colnames = similar_days_df.columns
    similar_days_df.columns = ([''] * (len(colnames)-12) +  ['similar'] * 12 ) + colnames 
    
    return(similar_days_df)


def create_zonal_NN_features_test(data,test_data,zone,similar_days=False) : 
    
    input_data_zone = data[data['zone'] == zone]
    test_data_zone = test_data[test_data['zone'] == zone]
    if similar_days :
        similar_days_df = find_all_similar_days_test(input_data_zone,test_data_zone)  
        similar_days_df = similar_days_df.rename(index=str, 
                                                 columns={'zone':'zone.1',
                                                          'year': 'year.1',
                                                          'weekday': 'weekday.1'})
        similar_days_df.index = range(similar_days_df.shape[0])
    
    ##make dummies
    hour_dummies = pd.get_dummies(test_data_zone["hour"])
    hour_dummies.columns = hour_dummies.columns = pd.Series(hour_dummies.columns).apply(lambda x : 'hour_' + str(x))

    month_dummies = pd.get_dummies(pd.concat([test_data_zone["month"],pd.Series(range(1,13))]))
    month_dummies.columns = month_dummies.columns = pd.Series(month_dummies.columns).apply(lambda x : 'month_' + str(x))
    
    month_dummies = month_dummies.iloc[0:(month_dummies.shape[0]-12)] 
    
    
    print(test_data_zone.shape)
    print(hour_dummies.shape)
    print(month_dummies.shape)
    
    test_data_zone = pd.concat([test_data_zone,hour_dummies,month_dummies],  axis=1) 
    test_data_zone.index = range(test_data_zone.shape[0])
    print('#####\n\n\n')
    print(test_data_zone.shape)
    print(similar_days_df.shape)
    
    if similar_days : 
        test_data_zone = pd.concat([test_data_zone,similar_days_df],axis=1)
    
    print('#################\n test_data_zone \n\n',test_data_zone.columns)
    test_data_zone = test_data_zone.drop(['id','hour','day','month', 'totDay'],axis =1 )
    
    return(test_data_zone)
     
def train_zonal_NN(input_data_zone,similar_days=False) :

    Y = np.array(input_data_zone['load'])
    Y = Y.reshape((Y.shape[0],1))
    X = np.array(input_data_zone.drop(['load','zone'],axis = 1)) 
    print(X.shape)
    X = X.reshape((X.shape[0],X.shape[1])) 
    print(X.shape)
    print(Y.shape)
    
    # create model
    model = Sequential()
    model.add(Dense(100, input_shape=(X.shape[1],), activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(100, activation='relu')) 
    model.add(Dense(1, activation='linear')) 
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

    # Fit the model
    model.fit(X, Y, epochs=20, batch_size=10) 
    
    return(model,X,Y)


def make_XY_test(data) :
    Y = np.array(data['load'])
    Y = Y.reshape((Y.shape[0],1))
    X = np.array(data.drop(['load','zone'],axis = 1)) 
    print(X.shape)
    X = X.reshape((X.shape[0],X.shape[1])) 
    print(X.shape)
    print(Y.shape)
    return(X,Y)
    

In [185]:
train_data = pd.read_csv(path + '/similar_days_df_z1.csv')
train_data = train_data.drop(['Unnamed: 0'],axis = 1)

for zone in range(10,21) :

    test_data = pd.read_csv(path + '/GEFCOM/temp_for_solutions.csv')
    test_data = create_zonal_NN_features_test(data,test_data,zone,similar_days=True)
    test_data = test_data[train_data.columns]
    test_data.to_csv('test_data_z' + str(zone) + '.csv')
 
 
X,Y = make_XY_test(test_data) 
pred = model.predict(X) 


2017-11-20 20:16:32.730078 0
(1344, 20)
(1344, 24)
(1344, 12)
#####



(1344, 56)
(1344, 20)
#################
 test_data_zone 

 Index(['id', 'zone', 'year', 'month', 'day', 'totDay', 'hour', 'weekday', 'w1',
       'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'load',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'id',
       'zone.1', 'year.1', 'month', 'day', 'totDay', 'hour', 'weekday.1',
       'similarw1', 'similarw2', 'similarw3', 'similarw4', 'similarw5',
       'similarw6', 'similarw7', 'similarw8', 'similarw9', 'similarw10',
       'similarw11', 'similarload'],
      dtype='object'

(1344, 20)
(1344, 24)
(1344, 12)
#####



(1344, 56)
(1344, 20)
#################
 test_data_zone 

 Index(['id', 'zone', 'year', 'month', 'day', 'totDay', 'hour', 'weekday', 'w1',
       'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'load',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'id',
       'zone.1', 'year.1', 'month', 'day', 'totDay', 'hour', 'weekday.1',
       'similarw1', 'similarw2', 'similarw3', 'similarw4', 'similarw5',
       'similarw6', 'similarw7', 'similarw8', 'similarw9', 'similarw10',
       'similarw11', 'similarload'],
      dtype='object')
2017-11-20 20:17:32.426608 

ValueError: Error when checking : expected dense_76_input to have shape (None, 65) but got array with shape (1344, 64)

In [ ]:
models ={}

In [187]:
for zone in range(10,21) :
    print(datetime.now())
    train_data = pd.read_csv(path + '/similar_days_df_z'+str(zone)+'.csv') 
    
    model,X1,Y1 = train_zonal_NN(train_data,similar_days=False)
    models[zone] = model 
    
    test_data = pd.read_csv(path + '/test_data_z'+str(zone)+'.csv') 
    
    X,Y = make_XY_test(test_data) 
    
    pred = model.predict(X) 
    actual_series = pd.Series(np.reshape(Y,(Y.size,)))
    predicted_series = pd.Series(np.reshape(pred,(pred.size,)))
    predicted_df = pd.DataFrame({'predicted_NN':predicted_series,
                                'actual' : actual_series})
    predicted_df.index = range(predicted_df.shape[0])  

    d = pd.read_csv(path + '/GEFCOM/temp_for_solutions.csv')
    d = d[d['zone'] == zone]
    d.index = range(d.shape[0]) 

    df = pd.concat([d[['zone','year','month','day','hour']],predicted_df],axis=1) 
    
    df.to_csv('predicted_results_z' + str(zone) + '.csv')
    

2017-11-20 20:18:24.971268
(38070, 65)
(38070, 65)
(38070, 1)
Epoch 1/20
38070/38070 [==============================] - 7s 196us/step - loss: 27916534.0503 - acc: 1.8387e-04
Epoch 2/20
38070/38070 [==============================] - 7s 174us/step - loss: 25091549.9765 - acc: 1.8387e-04
Epoch 3/20
38070/38070 [==============================] - 7s 185us/step - loss: 24599026.2846 - acc: 5.2535e-05
Epoch 4/20
38070/38070 [==============================] - 7s 191us/step - loss: 24465948.8806 - acc: 2.1014e-04
Epoch 5/20
38070/38070 [==============================] - 7s 178us/step - loss: 24319441.5398 - acc: 1.5760e-04
Epoch 6/20
38070/38070 [==============================] - 7s 175us/step - loss: 23794816.5868 - acc: 1.8387e-04
Epoch 7/20
38070/38070 [==============================] - 7s 174us/step - loss: 23763708.3508 - acc: 1.0507e-04
Epoch 8/20
38070/38070 [==============================] - 7s 174us/step - loss: 23597083.1916 - acc: 1.5760e-04
Epoch 9/20
38070/38070 [==================

38070/38070 [==============================] - 7s 182us/step - loss: 4236976.3267 - acc: 2.1014e-04
Epoch 11/20
38070/38070 [==============================] - 7s 181us/step - loss: 4190277.5902 - acc: 3.1521e-04
Epoch 12/20
38070/38070 [==============================] - 7s 180us/step - loss: 4124656.5923 - acc: 2.1014e-04
Epoch 13/20
38070/38070 [==============================] - 7s 180us/step - loss: 4089995.0393 - acc: 2.1014e-04
Epoch 14/20
38070/38070 [==============================] - 7s 180us/step - loss: 4059017.5809 - acc: 1.8387e-04
Epoch 15/20
38070/38070 [==============================] - 7s 179us/step - loss: 4050403.6772 - acc: 2.6267e-04
Epoch 16/20
38070/38070 [==============================] - 7s 186us/step - loss: 4017468.1101 - acc: 3.1521e-04
Epoch 17/20
38070/38070 [==============================] - 7s 181us/step - loss: 4018446.5725 - acc: 2.6267e-04
Epoch 18/20
38070/38070 [==============================] - 7s 185us/step - loss: 3981784.8506 - acc: 4.2028e-04
Epoc

Epoch 9/20
38070/38070 [==============================] - 7s 186us/step - loss: 68772018.9932 - acc: 2.6267e-05
Epoch 10/20
38070/38070 [==============================] - 7s 183us/step - loss: 68888662.9595 - acc: 5.2535e-05
Epoch 11/20
38070/38070 [==============================] - 7s 184us/step - loss: 68696797.9202 - acc: 7.8802e-05
Epoch 12/20
38070/38070 [==============================] - 7s 182us/step - loss: 68155934.6533 - acc: 1.0507e-04
Epoch 13/20
38070/38070 [==============================] - 7s 182us/step - loss: 68064537.9138 - acc: 2.6267e-05
Epoch 14/20
38070/38070 [==============================] - 7s 182us/step - loss: 67886012.3964 - acc: 7.8802e-05
Epoch 15/20
38070/38070 [==============================] - 7s 182us/step - loss: 68057996.2264 - acc: 5.2535e-05
Epoch 16/20
38070/38070 [==============================] - 7s 185us/step - loss: 67586787.1348 - acc: 0.0000e+00
Epoch 17/20
38070/38070 [==============================] - 7s 181us/step - loss: 67686628.4308 - 

In [157]:
input_data_zone = pd.read_csv('similar_days_df_z1.csv')
input_data_zone = input_data_zone.drop(['Unnamed: 0'],axis = 1)
model,X,Y = train_zonal_NN(input_data_zone,similar_days=True)



(38070, 64)
(38070, 64)
(38070, 1)
Epoch 1/20
18380/38070 [=============>................] - ETA: 3s - loss: 6852658.1634 - acc: 3.8085e-04

KeyboardInterrupt: 

In [188]:
models

{2: <keras.models.Sequential at 0x1a1a070358>,
 3: <keras.models.Sequential at 0x1a1a269e48>,
 4: <keras.models.Sequential at 0x1a1b533ac8>,
 5: <keras.models.Sequential at 0x1a1c929a90>,
 6: <keras.models.Sequential at 0x1a1d34eda0>,
 7: <keras.models.Sequential at 0x1a1de8c7b8>,
 8: <keras.models.Sequential at 0x1a22062780>,
 9: <keras.models.Sequential at 0x1a1e58cb38>,
 10: <keras.models.Sequential at 0x1a233a5978>,
 11: <keras.models.Sequential at 0x1a2e37e6a0>,
 12: <keras.models.Sequential at 0x1a2d4b0e48>,
 13: <keras.models.Sequential at 0x1a2da91e10>,
 14: <keras.models.Sequential at 0x1a1e597ef0>,
 15: <keras.models.Sequential at 0x1a2e0824a8>,
 16: <keras.models.Sequential at 0x1a2df65470>,
 17: <keras.models.Sequential at 0x1a2f48e438>,
 18: <keras.models.Sequential at 0x1a2fa74518>,
 19: <keras.models.Sequential at 0x1a3106ee48>,
 20: <keras.models.Sequential at 0x1a31ab3e10>}

In [8]:
### read in the solution loads
solution_loads = pd.read_csv(path + '/GEFCOM/Load_solution.csv')
solution_loads = pd.melt(solution_loads, id_vars=['id','zone_id','year','month','day','weight'],
        value_vars=np.core.defchararray.add(np.array(['h']*24),np.array(range(1,25)).astype('str')))
solution_loads = solution_loads.rename(index=str, columns={'value':'load','variable':'hour','zone_id':'zone'})
solution_loads['hour'] = solution_loads['hour'].apply(lambda x : x[1:])
solution_loads['hour'] = solution_loads['hour'].apply(lambda x : int(x))

In [ ]:
solution_data = pd.merge(left = solution_loads,right = solution_temps, how = 'inner',
                         on = ['year','month','day','hour'])

datetimes = pd.to_datetime((solution_data.year*10000+solution_data.month*100+solution_data.day).apply(str),format='%Y%m%d')
solution_data['weekday'] = datetimes.apply(lambda x : int(x.weekday() < 5))

datetimes = pd.to_datetime((2004*10000+solution_data.month*100+solution_data.day).apply(str),format='%Y%m%d')
solution_data['totDay'] = (datetimes - datetime(2004,1,1)).apply(lambda x : x.days)

solution_data = solution_data[list(input_data_zone.columns)]

solution_data['hour'] = solution_data['hour'].apply(lambda x: x -1)


(38070, 64)

In [181]:
df.to_csv('predicted_results_z' + str(zone) + '.csv')